In [2]:
%matplotlib inline


In [ ]:
from src.common_utils.custom_vars import InterferometerType


sampling_options_schema = {
    "opds": {
        "num": 51,
        "step": 0.2,
    },
    "wavenumbers": {
        "start": 1.,
        "stop": 2.5,
        "override_harmonic_order": -1,
    },
    "interferometer": {
        "device_type": InterferometerType.MICHELSON,
        "reflectance_scalar": 0.5,
    },
}
